In [2]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")

Main.FinancialBVAR

In [16]:
data_df = DataFrame(XLSX.readdata("shiller_data.xlsx", "Data!A8:N1852"), :auto)
first(data_df,20)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Date,P,D,E,CPI,Fraction,Rate GS10,missing,Price,Dividend,Price,Earnings,Earnings,CAPE
2,1871.01,4.44,0.26,0.4,12.4641,1871.04,5.32,missing,111.945,6.55534,111.945,10.0851,10.0851,NA
3,1871.02,4.5,0.26,0.4,12.8446,1871.12,5.32333,missing,110.096,6.36111,110.626,9.78632,9.83344,NA
4,1871.03,4.61,0.26,0.4,13.035,1871.21,5.32667,missing,111.14,6.26823,112.2,9.64343,9.7354,NA
5,1871.04,4.74,0.26,0.4,12.5592,1871.29,5.33,missing,118.603,6.50567,120.282,10.0087,10.1504,NA
6,1871.05,4.86,0.26,0.4,12.2738,1871.37,5.33333,missing,124.434,6.65695,126.757,10.2415,10.4327,NA
7,1871.06,4.82,0.26,0.4,12.0835,1871.46,5.33667,missing,125.354,6.76181,128.268,10.4028,10.6447,NA
8,1871.07,4.73,0.26,0.4,12.0835,1871.54,5.34,missing,123.013,6.76181,126.45,10.4028,10.6934,NA
9,1871.08,4.79,0.26,0.4,11.8932,1871.62,5.34333,missing,126.566,6.86997,130.691,10.5692,10.9136,NA


In [17]:
last(data_df,20)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,2023.01,3960.66,67.35,173.557,299.17,2023.04,3.53,4.54,4160.36,70.7459,2.70958e6,182.308,1.18734e5,28.3348
2,2023.02,4079.68,67.78,174.363,300.84,2023.12,3.75,4.65,4261.6,70.8023,2.77936e6,182.138,1.18788e5,28.9198
3,2023.03,3968.56,68.21,175.17,301.836,2023.21,3.66,4.69,4131.84,71.0164,2.69859e6,182.377,1.19114e5,27.9381
4,2023.04,4121.47,68.3767,177.117,303.363,2023.29,3.46,4.92,4269.44,70.8316,2.79232e6,183.476,1.19997e5,28.7648
5,2023.05,4146.17,68.5433,179.063,304.127,2023.37,3.57,5.14,4284.24,70.8258,2.80586e6,185.026,1.21178e5,28.7621
6,2023.06,4345.37,68.71,181.01,305.109,2023.46,3.75,5.16,4475.62,70.7696,2.93506e6,186.436,1.22262e5,29.94
7,2023.07,4508.08,68.911,182.09,305.691,2023.54,3.9,5.25,4634.36,70.8415,3.04303e6,187.191,122914.0,30.8917
8,2023.08,4457.36,69.1121,183.17,307.026,2023.62,4.17,5.3,4562.3,70.7392,2.99958e6,187.482,1.23264e5,30.3047
9,2023.09,4409.09,69.3131,184.25,307.789,2023.71,4.38,5.32,4501.71,70.7691,2.96363e6,188.12,1.23846e5,29.7997


In [4]:
columns = [:Date, :short_us, :real_us, :term_us]
data_df = DataFrame(XLSX.readtable("shiller_data.xlsx", "Data"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
macro_data = TimeArray(data_df; timestamp = :Date)

ArgumentError: ArgumentError: column name :Date not found in the data frame